In [17]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import re
import numpy as np
import pandas as pd
import chart_studio.plotly as py
import plotly.graph_objs as go
import itertools
from scipy import stats
from ast import literal_eval
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split


In [18]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Input, Activation, GlobalAveragePooling1D, Flatten, Concatenate, Conv1D, MaxPooling1D,Bidirectional,TimeDistributed,Reshape,Conv2D,MaxPool2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adam
from tensorflow.keras.preprocessing.text import one_hot, text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model

import fnmatch

import warnings

import string
from pathlib import Path
from random import shuffle
from ast import literal_eval

warnings.filterwarnings('ignore')

In [19]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Srijha
[nltk_data]     Kalyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Srijha
[nltk_data]     Kalyan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Srijha
[nltk_data]     Kalyan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
from nltk.stem.wordnet import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [21]:
import gensim
import json

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model
import tensorflow.keras.utils
from tensorflow.keras import utils as np_utils


In [22]:
WINDOWS_SIZE = 10
labels=['none','mild','moderate','moderately severe', 'severe']
num_classes = len(labels)

In [23]:
def transcripts_to_dataframe(directory):
    rows_list = []
        
    filenames = os.listdir(directory)
    
    if ".DS_Store" in filenames:
        filenames.remove(".DS_Store")
        
    for filename in filenames:
        transcript_path = os.path.join(directory, filename)
        transcript = pd.read_csv(transcript_path, sep='\t')
        m = re.search("(\d{3})_TRANSCRIPT.csv", filename)
        if m:
            person_id = m.group(1)
            p = {}
            question = ""
            answer = ""
            lines = len(transcript)
            for i in range(0, lines):
                row = transcript.iloc[i]
                if (row["speaker"] == "Ellie") or (i == lines - 1):
                    p["personId"] = person_id
                    if "(" in str(question):
                        question = question[question.index("(") + 1:question.index(")")]
                    p["question"] = question
                    p["answer"] = answer
                    if question != "":
                        rows_list.append(p)
                    p = {}
                    answer = ""
                    question = row["value"]
                else:
                    answer = str(answer) + " " + str(row["value"])

    all_participants = pd.DataFrame(rows_list, columns=['personId', 'question', 'answer'])
    all_participants.to_csv(directory + 'all.csv', sep=',')
    print("File was created")
    return all_participants

In [24]:
#loading the data
data_path = "transcripts/"
all_participants = transcripts_to_dataframe(data_path)

File was created


In [25]:
all_participants.head(20)

,personId,question,answer
0,300,hi i'm ellie thanks for coming in today,
1,300,i was created to talk to people in a safe and ...,
2,300,think of me as a friend i don't judge i can't ...,
3,300,i'm here to learn about people and would love ...,
4,300,i'll ask a few questions to get us started and...,
5,300,how are you doing today,good
6,300,that's good,
7,300,where are you from originally,atlanta georgia
8,300,really,
9,300,why'd you move to l_a,um my parents are from here um


In [26]:
#https://www.kaggle.com/currie32/the-importance-of-cleaning-text

def text_to_wordlist(text, remove_stopwords=True, stem_words=False):    
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = stopwords.words("english")
        text = [wordnet_lemmatizer.lemmatize(w) for w in text if not w in stops ]
        text = [w for w in text if w != "nan" ]
    else:
        text = [wordnet_lemmatizer.lemmatize(w) for w in text]
        text = [w for w in text if w != "nan" ]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    
    text = re.sub(r"\<", " ", text)
    text = re.sub(r"\>", " ", text)
    
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [27]:
#creating a corpus with the words from the answers without stopwords given by the patients
all_participants_mix = all_participants.copy() # However, if you need the original list unchanged when the new list is modified, you can use copy() method. This is called shallow copy.
all_participants_mix["answer"] = all_participants_mix.apply(lambda row: text_to_wordlist(row.answer).split(), axis=1)

In [28]:
words = [w for w in all_participants_mix['answer'].tolist()]
words = set(itertools.chain(*words)) #chain('ABC', 'DEF') --> A B C D E F
vocab_size = len(words)

In [29]:
windows_size = WINDOWS_SIZE
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(all_participants_mix['answer']) # fit_on_texts creates the vocabulary index based on word frequency.
#The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 1; word_index["cat"] = 2 it is word -> index dictionary so every word gets a unique integer value. 0 is reserved for padding. So lower integer means more frequent word 

tokenizer.fit_on_sequences(all_participants_mix['answer']) #texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary.

all_participants_mix['t_answer'] = tokenizer.texts_to_sequences(all_participants_mix['answer'])
all_participants_mix.head(15)

#   why are the output as numbers when text_to_sequences is called?
# the Tokenizer stores everything in the word_index during fit_on_texts. Then, when calling the texts_to_sequences method, only the top num_words are considered.

,personId,question,answer,t_answer
0,300,hi i'm ellie thanks for coming in today,[],[]
1,300,i was created to talk to people in a safe and ...,[],[]
2,300,think of me as a friend i don't judge i can't ...,[],[]
3,300,i'm here to learn about people and would love ...,[],[]
4,300,i'll ask a few questions to get us started and...,[],[]
5,300,how are you doing today,[good],[16]
6,300,that's good,[],[]
7,300,where are you from originally,"[atlanta, georgia]","[1634, 1997]"
8,300,really,[],[]
9,300,why'd you move to l_a,"[um, parent, um]","[1, 131, 1]"


In [30]:
phrases_lp = pd.read_csv('phrases_lp.csv', sep='\t', converters={"t_answer": literal_eval}) 
phrases_lp.head(15)


,Unnamed: 0,personId,answer,t_answer
0,0,300,"['good', 'atlanta', 'georgia', 'um', 'parent',...","[16, 1634, 1997, 1, 131, 1, 63, 5, 142, 5]"
1,1,300,"['atlanta', 'georgia', 'um', 'parent', 'um', '...","[1634, 1997, 1, 131, 1, 63, 5, 142, 5, 334]"
2,2,300,"['georgia', 'um', 'parent', 'um', 'love', 'lik...","[1997, 1, 131, 1, 63, 5, 142, 5, 334, 1]"
3,3,300,"['um', 'parent', 'um', 'love', 'like', 'weathe...","[1, 131, 1, 63, 5, 142, 5, 334, 1, 39]"
4,4,300,"['parent', 'um', 'love', 'like', 'weather', 'l...","[131, 1, 63, 5, 142, 5, 334, 1, 39, 1]"
5,5,300,"['um', 'love', 'like', 'weather', 'like', 'opp...","[1, 63, 5, 142, 5, 334, 1, 39, 1, 154]"
6,6,300,"['love', 'like', 'weather', 'like', 'opportuni...","[63, 5, 142, 5, 334, 1, 39, 1, 154, 527]"
7,7,300,"['like', 'weather', 'like', 'opportunity', 'um...","[5, 142, 5, 334, 1, 39, 1, 154, 527, 608]"
8,8,300,"['weather', 'like', 'opportunity', 'um', 'yes'...","[142, 5, 334, 1, 39, 1, 154, 527, 608, 100]"
9,9,300,"['like', 'opportunity', 'um', 'yes', 'um', 'to...","[5, 334, 1, 39, 1, 154, 527, 608, 100, 1998]"


In [31]:
from tensorflow.keras.utils import to_categorical
def load_avec_dataset_file(path,score_column):
    ds = pd.read_csv(path, sep=',')
    ds['level'] = pd.cut(ds[score_column], bins=[-1,0,5,10,15,25], labels=[0,1,2,3,4])  #cut function used to segregate array into bins 5 levels - 'none','mild','moderate','moderately severe', 'severe'
    ds['PHQ8_Score'] = ds[score_column]
    ds['cat_level'] = to_categorical(ds['level'], num_classes).tolist() #categorical levels 
    ds = ds[['Participant_ID', 'level', 'cat_level', 'PHQ8_Score','Gender']] 
    ds = ds.astype({"Participant_ID": float, "level": int, 'PHQ8_Score': int})
    return ds

In [32]:
train = load_avec_dataset_file('train_split_Depression_AVEC2017 (1).csv','PHQ8_Score')
dev = load_avec_dataset_file('dev_split_Depression_AVEC2017.csv','PHQ8_Score')
test = load_avec_dataset_file('full_test_split.csv','PHQ8_Score')
print("Size: train= {}, dev= {}, test={}".format(len(train), len(dev), len(test)))
train.head()

Size: train= 107, dev= 35, test=47


,Participant_ID,level,cat_level,PHQ8_Score,Gender
0,303.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,0
1,304.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",6,0
2,305.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",7,1
3,310.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",4,1
4,312.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",2,1


In [33]:
ds_total = pd.concat([dev,train,test])
total_phq8 = len(ds_total)
print("Total size = {}".format(total_phq8))

Total size = 189


In [34]:
ds_total.to_csv('ds_total.csv', sep='\t')
print("File was created")

File was created


In [35]:
def split_by_phq_level(ds):
    none_ds = ds[ds['level']==0]
    mild_ds = ds[ds['level']==1]
    moderate_ds = ds[ds['level']==2]
    moderate_severe_ds = ds[ds['level']==3]
    severe_ds = ds[ds['level']==4]
    return (none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds)

In [36]:
none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds = split_by_phq_level(ds_total)
print("Quantity per none_ds: {}, mild_ds: {}, moderate_ds {}, moderate_severe_ds: {}, severe_ds {}".format(len(none_ds), len(mild_ds), len(moderate_ds), len(moderate_severe_ds), len(severe_ds)))

Quantity per none_ds: 26, mild_ds: 70, moderate_ds 47, moderate_severe_ds: 24, severe_ds 22


In [37]:
b_none_ds = ds_total[ds_total['level']==0]
b_mild_ds = ds_total[ds_total['level']==1].sample(26)
b_moderate_ds = ds_total[ds_total['level']==2].sample(26)
b_moderate_severe_ds = ds_total[ds_total['level']==3]
b_severe_ds = ds_total[ds_total['level']==4]

ds_total_b = pd.concat([b_none_ds, b_mild_ds, b_moderate_ds, b_moderate_severe_ds, b_severe_ds])

In [38]:
ds_= ds_total_b.to_csv('ds_total_b.csv')

In [39]:
ds_lp = pd.merge(ds_total, phrases_lp,left_on='Participant_ID', right_on='personId')
ds_lp.drop(ds_lp[ds_lp["t_answer"].map(len) < 10].index, inplace = True)
ds_lp_b = pd.merge(ds_total_b, phrases_lp,left_on='Participant_ID', right_on='personId')
ds_lp_b.drop(ds_lp_b[ds_lp_b["t_answer"].map(len) < 10].index, inplace = True)

In [40]:
def distribute_instances(ds, split_in = [70,14,16]):
    ds_shuffled = ds.sample(frac=1)
    none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds = split_by_phq_level(ds_shuffled)
    eq_ds = dict()
    prev_none = prev_mild = prev_moderate = prev_moderate_severe = prev_severe = 0
    split = split_in
    for p in split:
        last_none = min(len(none_ds), prev_none + round(len(none_ds) * p/100))
        last_mild = min(len(mild_ds), prev_mild + round(len(mild_ds) * p/100))
        last_moderate = min(len(moderate_ds), prev_moderate + round(len(moderate_ds) * p/100))
        last_moderate_severe = min(len(moderate_severe_ds), prev_moderate_severe + round(len(moderate_severe_ds) * p/100))
        last_severe = min(len(severe_ds), prev_severe + round(len(severe_ds) * p/100))  
        eq_ds['d'+str(p)] = pd.concat([none_ds[prev_none: last_none], mild_ds[prev_mild: last_mild], moderate_ds[prev_moderate: last_moderate], moderate_severe_ds[prev_moderate_severe: last_moderate_severe], severe_ds[prev_severe: last_severe]])
        prev_none = last_none
        prev_mild = last_mild
        prev_moderate = last_moderate
        prev_moderate_severe = last_moderate_severe
        prev_severe = last_severe  
    return (eq_ds['d70'], eq_ds['d14'], eq_ds['d16'])

In [41]:
train_lp, dev_lp, test_lp = distribute_instances(ds_lp)
train_lp_b, dev_lp_b, test_lp_b = distribute_instances(ds_lp_b)

In [42]:
embeddings_index = dict()
f = open('glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


In [43]:
def fill_embedding_matrix(tokenizer):
    vocab_size = len(tokenizer.word_index) # tokenizer.word_index is the list that consist of all the unique words
    embedding_matrix = np.zeros((vocab_size+1, 100)) # creating an embedding matrix
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:        
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [44]:
embedding_matrix_lp = fill_embedding_matrix(tokenizer)
embedding_matrix_lp.shape

(7374, 100)

In [45]:
train_lp['t_answer']
train_lp.head()

,Participant_ID,level,cat_level,PHQ8_Score,Gender,Unnamed: 0,personId,answer,t_answer
95039,463.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,1,121679,463,"['saleing', 'family', 'pretty', 'fun', 'going'...","[6855, 61, 23, 127, 40, 38, 1441, 1238, 616, 2]"
126186,411.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,0,82912,411,"['incredible', 'healing', 'thing', 'believe', ...","[1129, 3866, 9, 343, 2, 1, 2, 12, 553, 944]"
58187,370.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,1,51323,370,"['even', 'though', 'could', 'get', 'full', 'ei...","[78, 217, 37, 18, 705, 363, 213, 2257, 77, 50]"
124592,408.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,0,80014,408,"['cannot', 'think', 'sorry', 'laughter', 'guil...","[68, 12, 374, 8, 305, 12, 70, 28, 305, 1177]"
115800,361.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,1,40165,361,"['like', 'kid', 'candy', 'store', 'get', 'play...","[5, 116, 2941, 857, 18, 288, 38, 4279, 1, 7]"


In [46]:
train_a =np.stack(train_lp['t_answer'])
dev_a = np.stack(dev_lp['t_answer'])
train_y = np.stack(train_lp['cat_level'], axis=0)
dev_y = np.stack(dev_lp['cat_level'], axis=0)

In [47]:
train_a_b = np.stack(train_lp_b['t_answer'], axis=0)
dev_a_b = np.stack(dev_lp_b['t_answer'], axis=0)
train_y_b = np.stack(train_lp_b['cat_level'], axis=0)
dev_y_b = np.stack(dev_lp_b['cat_level'], axis=0)

In [48]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [49]:
def plot_acc(history, title="Model Accuracy"):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()
    
def plot_loss(history, title="Model Loss"):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper right')
    plt.show()
    
def plot_compare_losses(history1, history2, name1="Red 1", name2="Red 2", title="Graph title"):
    plt.plot(history1.history['loss'], color="green")
    plt.plot(history1.history['val_loss'], 'r--', color="green")
    plt.plot(history2.history['loss'], color="blue")
    plt.plot(history2.history['val_loss'], 'r--', color="blue")
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train ' + name1, 'Val ' + name1, 
                'Train ' + name2, 'Val ' + name2],
               loc='upper right')
    plt.show()
    
def plot_compare_accs(history1, history2, name1="Red 1",
                      name2="Red 2", title="Graph title"):
    """Compara accuracies de dos entrenamientos con nombres name1 y name2"""
    plt.plot(history1.history['acc'], color="green")
    plt.plot(history1.history['val_acc'], 'r--', color="green")
    plt.plot(history2.history['acc'], color="blue")
    plt.plot(history2.history['val_acc'], 'r--', color="blue")
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train ' + name1, 'Val ' + name1, 
                'Train ' + name2, 'Val ' + name2], 
               loc='lower right')
    plt.show()

def plot_compare_multiple_metrics(history_array, names, colors, title="Graph title", metric='acc'):  
    legend = []
    for i in range(0, len(history_array)):
        plt.plot(history_array[i].history[metric], color=colors[i])
        plt.plot(history_array[i].history['val_' + metric], 'r--', color=colors[i])
        legend.append('Train ' + names[i])
        legend.append('Val ' + names[i])
    
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')   
    plt.axis
    plt.legend(legend, 
               loc='lower right')
    plt.show()

In [50]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [51]:
answer_inp = Input(shape=(windows_size, ))
embedding_size_glove = 100
answer_emb1 = Embedding(vocab_size+1, embedding_size_glove, weights=[embedding_matrix_lp], input_length=windows_size, trainable=False)(answer_inp)



In [52]:
bt = BatchNormalization()(answer_emb1)
lstm = LSTM(embedding_size_glove, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(bt)

dense1 = Dense(units=256, activation="relu")(lstm)
dense2 = Dense(units=256, activation="relu")(dense1)

flatten = Flatten()(dense2)

out = Dense(5,  activation='softmax')(flatten)

model = Model(inputs=[answer_inp], outputs=[out])
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 10, 100)           737400    
_________________________________________________________________
batch_normalization (BatchNo (None, 10, 100)           400       
_________________________________________________________________
lstm (LSTM)                  (None, 10, 100)           80400     
_________________________________________________________________
dense (Dense)                (None, 10, 256)           25856     
_________________________________________________________________
dense_1 (Dense)              (None, 10, 256)           65792     
_________________________________________________________________
flatten (Flatten)            (None, 2560)              0     

In [53]:
model_glove_lstm_hist = model.fit(train_a, train_y, validation_data=(dev_a, dev_y), epochs=30, batch_size=64, shuffle=True, callbacks=[early_stopping])

Train on 98795 samples, validate on 19760 samples
Epoch 1/30
98795/98795 [==============================] - 42s 430us/sample - loss: 1.4518 - accuracy: 0.3713 - val_loss: 1.3246 - val_accuracy: 0.4396
Epoch 2/30
98795/98795 [==============================] - 37s 378us/sample - loss: 1.2637 - accuracy: 0.4742 - val_loss: 1.0869 - val_accuracy: 0.5647
Epoch 3/30
98795/98795 [==============================] - 38s 388us/sample - loss: 1.0989 - accuracy: 0.5550 - val_loss: 0.9280 - val_accuracy: 0.6346
Epoch 4/30
98795/98795 [==============================] - 37s 377us/sample - loss: 0.9874 - accuracy: 0.6067 - val_loss: 0.8028 - val_accuracy: 0.6962
Epoch 5/30
98795/98795 [==============================] - 39s 397us/sample - loss: 0.9080 - accuracy: 0.6431 - val_loss: 0.7263 - val_accuracy: 0.7285
Epoch 6/30
98795/98795 [==============================] - 40s 406us/sample - loss: 0.8435 - accuracy: 0.6696 - val_loss: 0.6364 - val_accuracy: 0.7648
Epoch 7/30
98795/98795 [====================

In [54]:
model.save('model_glove_lstm.h5')

In [55]:
model_glove_lstm_hist_b = model.fit(train_a_b, train_y_b, validation_data=(dev_a_b, dev_y_b), epochs=30, batch_size=64, shuffle=True, callbacks=[early_stopping])

Train on 64316 samples, validate on 12864 samples
Epoch 1/30
64316/64316 [==============================] - 25s 384us/sample - loss: 0.3857 - accuracy: 0.8607 - val_loss: 0.0740 - val_accuracy: 0.9815
Epoch 2/30
64316/64316 [==============================] - 25s 382us/sample - loss: 0.3376 - accuracy: 0.8788 - val_loss: 0.0669 - val_accuracy: 0.9842
Epoch 3/30
64316/64316 [==============================] - 24s 375us/sample - loss: 0.3120 - accuracy: 0.8879 - val_loss: 0.0599 - val_accuracy: 0.9845
Epoch 4/30
64316/64316 [==============================] - 24s 373us/sample - loss: 0.2871 - accuracy: 0.8972 - val_loss: 0.0559 - val_accuracy: 0.9854
Epoch 5/30
64316/64316 [==============================] - 24s 378us/sample - loss: 0.2786 - accuracy: 0.9013 - val_loss: 0.0551 - val_accuracy: 0.9854
Epoch 6/30
64316/64316 [==============================] - 25s 382us/sample - loss: 0.2646 - accuracy: 0.9046 - val_loss: 0.0542 - val_accuracy: 0.9852
Epoch 7/30
64316/64316 [====================

In [56]:
model.save('model_glove_lstm_b.h5')

In [57]:
answer_inp = Input(shape=(windows_size, ))
embedding_size_glove = 100
answer_emb1 = Embedding(vocab_size+1, embedding_size_glove, weights=[embedding_matrix_lp], input_length=windows_size, trainable=False)(answer_inp)


lstm1 = LSTM(embedding_size_glove, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(answer_emb1)
lstm2 = LSTM(embedding_size_glove, dropout=0.2, recurrent_dropout=0.2)(lstm1)

X = Dropout(0.2)(lstm2)
bt = BatchNormalization()(X)
dense1 = Dense(units=256, activation="relu")(bt)

out = Dense(5,  activation='softmax')(dense1)

model_2lstm = Model(inputs=[answer_inp], outputs=[out])
model_2lstm.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
model_2lstm.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 100)           737400    
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 100)           80400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2585

In [58]:
model_glove_2lstm_b_hist = model_2lstm.fit(train_a_b, train_y_b, validation_data=(dev_a_b, dev_y_b), epochs=30, batch_size=64, shuffle=True, callbacks=[early_stopping])

Train on 64316 samples, validate on 12864 samples
Epoch 1/30
64316/64316 [==============================] - 46s 720us/sample - loss: 1.6160 - accuracy: 0.2439 - val_loss: 1.5850 - val_accuracy: 0.2614
Epoch 2/30
64316/64316 [==============================] - 37s 569us/sample - loss: 1.5728 - accuracy: 0.2797 - val_loss: 1.5526 - val_accuracy: 0.2996
Epoch 3/30
64316/64316 [==============================] - 36s 566us/sample - loss: 1.5345 - accuracy: 0.3125 - val_loss: 1.4871 - val_accuracy: 0.3500
Epoch 4/30
64316/64316 [==============================] - 39s 601us/sample - loss: 1.4722 - accuracy: 0.3624 - val_loss: 1.3962 - val_accuracy: 0.4082
Epoch 5/30
64316/64316 [==============================] - 42s 651us/sample - loss: 1.3924 - accuracy: 0.4111 - val_loss: 1.2893 - val_accuracy: 0.4737
Epoch 6/30
64316/64316 [==============================] - 51s 799us/sample - loss: 1.3083 - accuracy: 0.4600 - val_loss: 1.1948 - val_accuracy: 0.5283
Epoch 7/30
64316/64316 [====================

In [59]:
model_2lstm.save('model_2lstm_b.h5')

In [60]:
# main model
answer_inp = Input(shape=(windows_size, ))
embedding_size_glove = 100
answer_emb1 = Embedding(vocab_size+1, embedding_size_glove, weights=[embedding_matrix_lp], input_length=windows_size, trainable=False)(answer_inp)


bi_lstm =  Bidirectional (LSTM (embedding_size_glove,return_sequences=True,dropout=0.50),merge_mode='concat')(answer_emb1)
model_bi1 = TimeDistributed(Dense(embedding_size_glove,activation='relu'))(bi_lstm) #TimeDistributed method is used to apply a Dense layer to each of the time-steps independently. We used Dropout and l2_reg regularizers to reduce overfitting.
model_bi2 = Flatten()(model_bi1)
model_bi3 = Dense(256,activation='relu')(model_bi2)
output = Dense(5,activation='softmax')(model_bi3)
model_bi = Model(answer_inp,output)
model_bi.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model_bi.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 10, 100)           737400    
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 200)           160800    
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 100)           20100     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1000)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               256256    
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 1285

In [61]:
model_glove_bilstm = model_bi.fit(train_a_b, train_y_b, validation_data=(dev_a_b, dev_y_b), epochs=30, batch_size=64, shuffle=True, callbacks=[early_stopping])

Train on 64316 samples, validate on 12864 samples
Epoch 1/30
64316/64316 [==============================] - 33s 508us/sample - loss: 1.5807 - accuracy: 0.2648 - val_loss: 1.5552 - val_accuracy: 0.2844
Epoch 2/30
64316/64316 [==============================] - 26s 397us/sample - loss: 1.5456 - accuracy: 0.2996 - val_loss: 1.5061 - val_accuracy: 0.3270
Epoch 3/30
64316/64316 [==============================] - 26s 399us/sample - loss: 1.4966 - accuracy: 0.3356 - val_loss: 1.4572 - val_accuracy: 0.3714
Epoch 4/30
64316/64316 [==============================] - 26s 402us/sample - loss: 1.4332 - accuracy: 0.3801 - val_loss: 1.3715 - val_accuracy: 0.4317
Epoch 5/30
64316/64316 [==============================] - 26s 401us/sample - loss: 1.3689 - accuracy: 0.4199 - val_loss: 1.2754 - val_accuracy: 0.4777
Epoch 6/30
64316/64316 [==============================] - 26s 401us/sample - loss: 1.3079 - accuracy: 0.4546 - val_loss: 1.1951 - val_accuracy: 0.5257
Epoch 7/30
64316/64316 [====================

In [62]:
model_bi.save('model_bilstm_a_b.h5')

In [63]:
from sklearn.metrics import roc_auc_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from tensorflow.keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.callbacks import Callback

In [64]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [65]:
# main model
answer_inp = Input(shape=(windows_size, ))
embedding_size_glove = 100
answer_emb1 = Embedding(vocab_size+1, embedding_size_glove, weights=[embedding_matrix_lp], input_length=windows_size, trainable=False)(answer_inp)

x = SpatialDropout1D(0.2)(answer_emb1)
x = Bidirectional(GRU(embedding_size_glove, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = concatenate([avg_pool, max_pool])
outp = Dense(5, activation="softmax")(conc)
    
model_gru = Model(inputs=answer_inp, outputs=outp)
model_gru.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_gru.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 10, 100)      737400      input_4[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 10, 100)      0           embedding_3[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 10, 200)      121200      spatial_dropout1d[0][0]          
____________________________________________________________________________________________

In [66]:
batch_size = 64
epochs = 30
RocAuc = RocAucEvaluation(validation_data=(dev_a,dev_y), interval=1)

In [67]:
hist = model_gru.fit(train_a, train_y, batch_size=batch_size, epochs=epochs, validation_data=(dev_a, dev_y),
                 callbacks=[RocAuc], verbose=2)

Train on 98795 samples, validate on 19760 samples
Epoch 1/30

 ROC-AUC - epoch: 1 - score: 0.672460 

98795/98795 - 37s - loss: 1.4750 - accuracy: 0.3645 - val_loss: 1.4270 - val_accuracy: 0.3881
Epoch 2/30

 ROC-AUC - epoch: 2 - score: 0.757883 

98795/98795 - 35s - loss: 1.3814 - accuracy: 0.4194 - val_loss: 1.3081 - val_accuracy: 0.4523
Epoch 3/30

 ROC-AUC - epoch: 3 - score: 0.818856 

98795/98795 - 33s - loss: 1.2566 - accuracy: 0.4887 - val_loss: 1.1584 - val_accuracy: 0.5530
Epoch 4/30

 ROC-AUC - epoch: 4 - score: 0.859144 

98795/98795 - 33s - loss: 1.1424 - accuracy: 0.5469 - val_loss: 1.0397 - val_accuracy: 0.6025
Epoch 5/30

 ROC-AUC - epoch: 5 - score: 0.884402 

98795/98795 - 33s - loss: 1.0448 - accuracy: 0.5936 - val_loss: 0.9622 - val_accuracy: 0.6352
Epoch 6/30

 ROC-AUC - epoch: 6 - score: 0.909178 

98795/98795 - 33s - loss: 0.9663 - accuracy: 0.6264 - val_loss: 0.8410 - val_accuracy: 0.6883
Epoch 7/30

 ROC-AUC - epoch: 7 - score: 0.922521 

98795/98795 - 34s - lo

In [68]:
model_gru.save('model_gru.h5')

In [69]:
filter_sizes = [1,2,3,5]
num_filters = 36
answer_inp = Input(shape=(windows_size, ))
embedding_size_glove = 100
answer_emb1 = Embedding(vocab_size+1, embedding_size_glove, weights=[embedding_matrix_lp], input_length=windows_size, trainable=False)(answer_inp)
x = Reshape((windows_size, embedding_size_glove, 1))(answer_emb1)
maxpool_pool = []
for i in range(len(filter_sizes)):
  conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embedding_size_glove),kernel_initializer='he_normal', activation='elu')(x)
  maxpool_pool.append(MaxPool2D(pool_size=(windows_size - filter_sizes[i] + 1, 1))(conv))

z = Concatenate(axis=1)(maxpool_pool)   
z = Flatten()(z)
z = Dropout(0.1)(z)

outp = Dense(5, activation="softmax")(z)

model = Model(inputs=answer_inp, outputs=outp)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 10, 100)      737400      input_5[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 10, 100, 1)   0           embedding_4[0][0]                
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 10, 1, 36)    3636        reshape[0][0]                    
____________________________________________________________________________________________

In [70]:
model_cnn =  model.fit(train_a, train_y, batch_size=64, epochs=30, validation_data=(dev_a, dev_y), verbose=2,callbacks=[early_stopping])

Train on 98795 samples, validate on 19760 samples
Epoch 1/30
98795/98795 - 8s - loss: 1.4535 - accuracy: 0.3803 - val_loss: 1.3552 - val_accuracy: 0.4138
Epoch 2/30
98795/98795 - 8s - loss: 1.2482 - accuracy: 0.4899 - val_loss: 1.2066 - val_accuracy: 0.5063
Epoch 3/30
98795/98795 - 9s - loss: 1.0916 - accuracy: 0.5653 - val_loss: 1.0926 - val_accuracy: 0.5616
Epoch 4/30
98795/98795 - 9s - loss: 0.9770 - accuracy: 0.6166 - val_loss: 1.0281 - val_accuracy: 0.5894
Epoch 5/30
98795/98795 - 9s - loss: 0.8971 - accuracy: 0.6503 - val_loss: 0.9745 - val_accuracy: 0.6221
Epoch 6/30
98795/98795 - 9s - loss: 0.8382 - accuracy: 0.6755 - val_loss: 0.8786 - val_accuracy: 0.6598
Epoch 7/30
98795/98795 - 9s - loss: 0.7913 - accuracy: 0.6955 - val_loss: 0.8387 - val_accuracy: 0.6808
Epoch 8/30
98795/98795 - 9s - loss: 0.7545 - accuracy: 0.7111 - val_loss: 0.8131 - val_accuracy: 0.6906
Epoch 9/30
98795/98795 - 9s - loss: 0.7238 - accuracy: 0.7227 - val_loss: 0.8084 - val_accuracy: 0.6938
Epoch 10/30
98

In [71]:
model.save('model_cnn.h5')

In [72]:
lstm_cnn = Sequential()
lstm_cnn.add(Embedding(vocab_size+1, 100,weights=[embedding_matrix_lp],input_length=windows_size, trainable=False))
lstm_cnn.add(Dropout(0.2))
lstm_cnn.add(Conv1D(64, 5, activation='relu'))
lstm_cnn.add(MaxPooling1D(pool_size=4))
lstm_cnn.add(LSTM(100))
lstm_cnn.add(Dense(5, activation='softmax'))
lstm_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
lstm_cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 10, 100)           737400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 100)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 6, 64)             32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 64)             0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 505       
Total params: 835,969
Trainable params: 98,569
Non-trainable params: 737,400
_____________________________________________

In [73]:
hist = lstm_cnn.fit(train_a_b, train_y_b, validation_data=(dev_a_b, dev_y_b), epochs=30, batch_size=64, shuffle=True, callbacks=[early_stopping])

Train on 64316 samples, validate on 12864 samples
Epoch 1/30
64316/64316 [==============================] - 8s 132us/sample - loss: 1.5772 - accuracy: 0.2717 - val_loss: 1.5561 - val_accuracy: 0.2950
Epoch 2/30
64316/64316 [==============================] - 7s 109us/sample - loss: 1.5218 - accuracy: 0.3209 - val_loss: 1.5106 - val_accuracy: 0.3305
Epoch 3/30
64316/64316 [==============================] - 7s 108us/sample - loss: 1.4552 - accuracy: 0.3693 - val_loss: 1.4500 - val_accuracy: 0.3727
Epoch 4/30
64316/64316 [==============================] - 7s 108us/sample - loss: 1.4037 - accuracy: 0.3996 - val_loss: 1.4132 - val_accuracy: 0.4007
Epoch 5/30
64316/64316 [==============================] - 7s 110us/sample - loss: 1.3633 - accuracy: 0.4226 - val_loss: 1.3710 - val_accuracy: 0.4261
Epoch 6/30
64316/64316 [==============================] - 7s 110us/sample - loss: 1.3329 - accuracy: 0.4396 - val_loss: 1.3698 - val_accuracy: 0.4250
Epoch 7/30
64316/64316 [==========================

In [74]:
lstm_cnn.save('model_lstm_cnn.h5')

In [ ]:
class AttentionLayer(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """
    def __init__(self, **kwargs):
        self.init = initializers.get('glorot_uniform')
        super(AttentionLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        assert len(input_shape) == 3
        
        self.W = self.add_weight(name='Attention_Weight',
                                 shape=(input_shape[-1], input_shape[-1]),
                                 initializer=self.init,
                                 trainable=True)
        self.b = self.add_weight(name='Attention_Bias',
                                 shape=(input_shape[-1], ),
                                 initializer=self.init,
                                 trainable=True)
        self.u = self.add_weight(name='Attention_Context_Vector',
                                 shape=(input_shape[-1], 1),
                                 initializer=self.init,
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
    
    def call(self, x):
        # refer to the original paper
        # link: https://www.cs.cmu.edu/~hovy/papers/16HLT-hierarchical-attention-networks.pdf
        
        u_it = K.tanh(K.dot(x, self.W) + self.b)
        
       #Through structure the vector applied  is used as
        #Make attention value into probability distribution through
        a_it = K.dot(u_it, self.u)
        a_it = K.squeeze(a_it, -1)
        a_it = K.softmax(a_it)
        
        return a_it
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1])


In [ ]:
answer_inp = Input(shape=(windows_size, ))
embedding_size_glove = 100
answer_emb1 = Embedding(vocab_size+1, embedding_size_glove, weights=[embedding_matrix_lp], input_length=windows_size, trainable=False)(answer_inp)

In [ ]:
lstm_dim=50
bilstm = Bidirectional(LSTM(lstm_dim, return_sequences=True))(answer_emb1)  #Input should be three dimensional
# bilstm = LSTM(2*lstm_dim, return_sequences=True)(input_data) 
# bilstm_output = Dense(1)(bilstm)

attention_layer = AttentionLayer()(bilstm)
print(attention_layer)
repeated_word_attention = RepeatVector(lstm_dim * 2)(attention_layer)
repeated_word_attention = Permute([2, 1])(repeated_word_attention)
sentence_representation = Multiply()([bilstm, repeated_word_attention])
sentence_representation = Lambda(lambda x: K.sum(x, axis=1))(sentence_representation) #total summation of the multiplied bilstm and attention 

bilstm_output = Dense(5,activation='softmax')(sentence_representation)

model = Model(inputs=[answer_inp],
            outputs=[bilstm_output])
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_a_b, train_y_b, validation_data=(dev_a_b, dev_y_b), epochs=30, batch_size=64, shuffle=True)

In [ ]:
model.save('model_bilstm_attn.h5')